In [8]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import pickle

# Hyperparameters
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 1000

device

'mps'

In [9]:
with open("x.txt", "r", encoding="UTF-8") as f:
    text = f.read()

chars = sorted(set(text))
vocab_size = len(chars)

FileNotFoundError: [Errno 2] No such file or directory: 'x.txt'

In [ ]:
str_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_str = { i:ch for i, ch in enumerate(chars) }
encode = lambda str: [str_to_int[ch] for ch in str]
decode = lambda ints: "".join([int_to_str[i] for i in ints])

data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))

    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            return logits, None
        
        # Reshape tensor to calculate loss
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]

            # Generate probabilities using logits and select a character based on that
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)

        return index

In [ ]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

# New line character to start
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, 500)[0].tolist())
print(generated_chars)

In [ ]:
@torch.no_grad
def estimate_loss():
    model.eval() # Set model to evaluation mode
    split = "val"
    losses = torch.zeros(eval_iters)

    for i in range(eval_iters):
        X, Y = get_batch(split)
        _, loss = model(X, Y) # Automatically calls forward method
        losses[i] = loss.item()
    
    model.train() # Set model back to training mode
    return losses.mean()


In [ ]:
# Training

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        print(f"Step: {iter}, Loss: {estimate_loss().item()}")

    xb, yb = get_batch("train")
    logits, loss = model.forward(xb, yb)

    optimizer.zero_grad(set_to_none=True) # Do not add previous gradients
    loss.backward() # Backpropogate loss
    optimizer.step() # Use the computed gradient to optimize model
 
print(loss.item())

In [ ]:
with open("bigram.pkl", "wb") as f:
    pickle.dump(model, f)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, 500)[0].tolist())
print(generated_chars)